In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
# read in data

datafile = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
test.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


In [4]:
species = datafile['species']
X_train = datafile.drop('species', axis=1)

In [5]:
# not looking at actual images for first pass, so drop id column from both test and train
X_train = X_train.drop('id', axis=1)
X_test = test.drop('id', axis=1)

In [6]:
# one hot encode species
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(species)
lb.classes_

array(['Acer_Capillipes', 'Acer_Circinatum', 'Acer_Mono', 'Acer_Opalus',
       'Acer_Palmatum', 'Acer_Pictum', 'Acer_Platanoids', 'Acer_Rubrum',
       'Acer_Rufinerve', 'Acer_Saccharinum', 'Alnus_Cordata',
       'Alnus_Maximowiczii', 'Alnus_Rubra', 'Alnus_Sieboldiana',
       'Alnus_Viridis', 'Arundinaria_Simonii', 'Betula_Austrosinensis',
       'Betula_Pendula', 'Callicarpa_Bodinieri', 'Castanea_Sativa',
       'Celtis_Koraiensis', 'Cercis_Siliquastrum', 'Cornus_Chinensis',
       'Cornus_Controversa', 'Cornus_Macrophylla', 'Cotinus_Coggygria',
       'Crataegus_Monogyna', 'Cytisus_Battandieri',
       'Eucalyptus_Glaucescens', 'Eucalyptus_Neglecta',
       'Eucalyptus_Urnigera', 'Fagus_Sylvatica', 'Ginkgo_Biloba',
       'Ilex_Aquifolium', 'Ilex_Cornuta', 'Liquidambar_Styraciflua',
       'Liriodendron_Tulipifera', 'Lithocarpus_Cleistocarpus',
       'Lithocarpus_Edulis', 'Magnolia_Heptapeta', 'Magnolia_Salicifolia',
       'Morus_Nigra', 'Olea_Europaea', 'Phildelphus', 'Populus_

In [8]:
# model
# start with random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [ ]:
forest = RandomForestClassifier(oob_score=True)

param_grid = {
    'n_estimators': [10, 30, 100, 300, 600]
}

model = GridSearchCV(forest, param_grid, cv=5)

model.fit(X_train, species)

print(model.score(X_train, species))
print(model.best_score_)
print(model.best_params_)

In [63]:
predictions = model.predict_proba(X_test)

In [96]:
predictions[:4]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.00333333,
         0.00333333,  0.        ,  0.        ,  0.        ,  0.        ,
         0.00333333,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.02333333,  0.        ,  0.00333333,  0.        ,
         0.        ,  0.00666667,  0.        ,  0.        ,  0.        ,
         0.        ,  0.00333333,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.00666667,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.00333333,  0.00333333,  0.        ,  0.00666667,
         0.        ,  0.09666667,  0.        ,  0.        ,  0.        ,
         0.        ,  0.46      ,  0.        ,  0.00333333,  0.00333333,
         0.01      ,  0.06666667,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.04666667,
         0.        ,  0.00333333,  0.        ,  0. 

In [106]:
labels = test['id']
headers = lb.classes_.tolist()
headers = ['id']+headers

In [99]:
results = pd.DataFrame(data=predictions)
labels = pd.DataFrame(labels)
results = pd.concat([labels, results], axis=1)

In [107]:
results.columns = headers

In [111]:
results.to_csv('results.csv', index=False)